In [23]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN

# 1. Read data
df = pd.read_csv("clustering_model_input.csv")

# 2. Extract lat-long
coords = df[["MedianLatitude", "MedianLongitude"]].to_numpy()

# 3. Convert degrees to radians (required for haversine)
coords_rad = np.radians(coords)

# 4. Hyperparameters for ~200m radius
EPS_KM = 0.2               # 200 metres
MIN_SAMPLES = 3            # at least 3 shops in that radius

eps_rad = EPS_KM / 6371.0  # ≈ 3.139e-05

# 5. Run DBSCAN
db = DBSCAN(
    eps=eps_rad,
    min_samples=MIN_SAMPLES,
    metric="haversine",
    n_jobs=-1
)

labels = db.fit_predict(coords_rad)

# 6. Attach cluster ids
df["cluster_id"] = labels

# 7. Save
df.to_csv("storeclutering_model_output_200m.csv", index=False)

# Optional: quick sanity check
# import numpy as np
# n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
# noise_pct = (np.sum(labels == -1) / len(labels)) * 100
# print("Clusters:", n_clusters, "Noise %:", round(noise_pct, 1))


### Keyword based clustering

In [25]:
import pandas as pd
import re
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS



In [27]:
df = pd.read_csv('keyword_store_input.csv', encoding = 'latin')

In [28]:
df.head()

LIS Store Code                      Store Name
0     KRBLGT_143067                        Uniavtar
1     KRBLGT_189820                   Haryana Fresh
2     KRBLGT_141706             Sodhi Supar Mart 83
3  Pidilite-MR-2564             D H Mart Chandivali
4  Pidilite-MR-2563  Ratna Super Market Mulund West

In [29]:
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    text = " ".join([word for word in text.split() if word not in ENGLISH_STOP_WORDS])
    return text

df['StoreName_Clean'] = df['Store Name'].apply(clean_text)

In [30]:
df.head()

LIS Store Code                      Store Name  \
0     KRBLGT_143067                        Uniavtar   
1     KRBLGT_189820                   Haryana Fresh   
2     KRBLGT_141706             Sodhi Supar Mart 83   
3  Pidilite-MR-2564             D H Mart Chandivali   
4  Pidilite-MR-2563  Ratna Super Market Mulund West   

                  StoreName_Clean  
0                        uniavtar  
1                   haryana fresh  
2             sodhi supar mart 83  
3             d h mart chandivali  
4  ratna super market mulund west

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=1000)  # top 50 keywords
X = vectorizer.fit_transform(df['StoreName_Clean'])
keywords = vectorizer.get_feature_names_out()
# print(keywords)

In [34]:
from sklearn.cluster import KMeans
import numpy as np

# keyword embeddings using TF-IDF values avg
keyword_vectors = X.toarray().mean(axis=0).reshape(-1,1)

kmeans = KMeans(n_clusters=5, random_state=42)
labels = kmeans.fit_predict(keyword_vectors)

keyword_category_df = pd.DataFrame({
    "keyword": keywords,
    "cluster": labels
})

print(keyword_category_df.sort_values("cluster"))


      keyword  cluster
635     patel        0
181      city        0
178    choice        0
832     shree        0
557       mkt        0
..        ...      ...
355      hill        4
356     hills        4
357  himalaya        4
344   hanuman        4
999      zone        4

[1000 rows x 2 columns]


In [35]:
def assign_category(text):
    for i, word in enumerate(keywords):
        if word in text:
            return f"Category_{labels[i]}"
    return "Others"

df['AutoCategory'] = df['StoreName_Clean'].apply(assign_category)


In [37]:
df.head(10)

LIS Store Code                       Store Name  \
0     KRBLGT_143067                         Uniavtar   
1     KRBLGT_189820                    Haryana Fresh   
2     KRBLGT_141706              Sodhi Supar Mart 83   
3  Pidilite-MR-2564              D H Mart Chandivali   
4  Pidilite-MR-2563   Ratna Super Market Mulund West   
5  Pidilite-MR-2299    One Stop Super Market Sanpada   
6  Pidilite-MR-2060  Solanki Super Market Dadar East   
7  Pidilite-MR-1694                 Jyothi Book Depo   
8  Pidilite-MR-1693               Kushi Super Market   
9  Pidilite-MR-1692              Mitraa Super Market   

                   StoreName_Clean AutoCategory  
0                         uniavtar   Category_4  
1                    haryana fresh   Category_4  
2              sodhi supar mart 83   Category_4  
3              d h mart chandivali   Category_4  
4   ratna super market mulund west   Category_4  
5        stop super market sanpada   Category_4  
6  solanki super market dadar east   Category_4  
7                 jyothi book depo   Category_4  
8               kushi super market   Category_4  
9              mitraa super market   Category_4

In [38]:
df.to_csv('keyword_output.csv')

In [40]:
import re
import numpy as np

def normalize(name):
    if pd.isna(name):
        return ""
    name = str(name).lower()
    name = re.sub(r'[^a-z0-9\s]', '', name)  # remove special chars
    remove_words = ["store", "mart", "pvt", "ltd", "india", "the", "shop"]
    name = " ".join([w for w in name.split() if w not in remove_words])
    return name.strip()

df["clean_store"] = df["Store Name"].apply(normalize)


In [43]:
pip install rapidfuzz?

Object `rapidfuzz` not found.


In [44]:
from rapidfuzz import process, fuzz

unique_stores = df["clean_store"].unique()
mapping = {}

for store in unique_stores:
    if store not in mapping:
        matches = process.extract(store, unique_stores, scorer=fuzz.token_sort_ratio)
        similar = [m for m, score, _ in matches if score >= 85]
        # assign group name
        for s in similar:
            mapping[s] = store

df["Grouped_Store"] = df["clean_store"].map(mapping)


In [45]:
df.head(10)

LIS Store Code                       Store Name  \
0     KRBLGT_143067                         Uniavtar   
1     KRBLGT_189820                    Haryana Fresh   
2     KRBLGT_141706              Sodhi Supar Mart 83   
3  Pidilite-MR-2564              D H Mart Chandivali   
4  Pidilite-MR-2563   Ratna Super Market Mulund West   
5  Pidilite-MR-2299    One Stop Super Market Sanpada   
6  Pidilite-MR-2060  Solanki Super Market Dadar East   
7  Pidilite-MR-1694                 Jyothi Book Depo   
8  Pidilite-MR-1693               Kushi Super Market   
9  Pidilite-MR-1692              Mitraa Super Market   

                   StoreName_Clean AutoCategory  \
0                         uniavtar   Category_4   
1                    haryana fresh   Category_4   
2              sodhi supar mart 83   Category_4   
3              d h mart chandivali   Category_4   
4   ratna super market mulund west   Category_4   
5        stop super market sanpada   Category_4   
6  solanki super market dadar east   Category_4   
7                 jyothi book depo   Category_4   
8               kushi super market   Category_4   
9              mitraa super market   Category_4   

                       clean_store                    Grouped_Store  
0                         uniavtar                         uniavtar  
1                    haryana fresh                    haryana fresh  
2                   sodhi supar 83                   sodhi supar 83  
3                   d h chandivali                   d h chandivali  
4   ratna super market mulund west   ratna super market mulund west  
5    one stop super market sanpada    one stop super market sanpada  
6  solanki super market dadar east  solanki super market dadar east  
7                 jyothi book depo                 jyothi book depo  
8               kushi super market              keshav super market  
9              mitraa super market              mitraa super market

In [46]:
df.to_csv('keyword_output.csv')

### latlong + ketyword based both

In [57]:
import pandas as pd

In [65]:
df = pd.read_csv('keyword_store_input.csv', encoding = 'latin')

In [67]:
df.head()

LIS Store Code                      Store Name     City   Latitude  \
0     KRBLGT_143067                        Uniavtar    Delhi  28.677668   
1     KRBLGT_189820                   Haryana Fresh    Delhi  28.524524   
2     KRBLGT_141706             Sodhi Supar Mart 83  Gurgaon  28.390209   
3  Pidilite-MR-2564             D H Mart Chandivali   Mumbai  19.118382   
4  Pidilite-MR-2563  Ratna Super Market Mulund West   Mumbai  19.137638   

   Longitude  
0  77.104838  
1  77.144927  
2  76.959405  
3  72.895984  
4  72.930074

In [69]:
import pandas as pd
import numpy as np
import re

from sklearn.cluster import DBSCAN
from rapidfuzz import fuzz

# =============================
# 1. NORMALIZE STORE NAMES & CITY
# =============================

def normalize_store(name):
    if pd.isna(name):
        return ""
    name = str(name).lower()
    name = re.sub(r'[^a-z0-9\s]', ' ', name)
    name = re.sub(r'\s+', ' ', name).strip()

    remove_words = [
        "store", "mart", "pvt", "ltd", "india", "the", "shop",
        "supermarket", "hypermarket", "format"
    ]
    tokens = [w for w in name.split() if w not in remove_words]
    return " ".join(tokens)

def normalize_city(city):
    if pd.isna(city):
        return ""
    city = str(city).lower()
    city = re.sub(r'[^a-z0-9\s]', ' ', city)
    city = re.sub(r'\s+', ' ', city).strip()
    return city

df["clean_name"] = df["Store Name"].apply(normalize_store)
df["clean_city"] = df["City"].apply(normalize_city)

# =============================
# 2. CITY-WISE DBSCAN ON LAT-LONG
# =============================

df["city_loc_cluster"] = -1   # combined city+location cluster id
cluster_counter = 0           # global counter for unique cluster IDs

eps = 0.2 / 6371.0   # 200 meters in radians (Earth radius ~6371km)

for city in df["clean_city"].unique():
    sub_idx = df["clean_city"] == city
    sub = df.loc[sub_idx]

    # If lat/long missing, skip DBSCAN for those
    coords = sub[["Latitude", "Longitude"]].values

    # handle rows with NaNs in lat/long
    valid_mask = ~np.isnan(coords).any(axis=1)
    valid_idx = sub[valid_mask].index
    invalid_idx = sub[~valid_mask].index

    # For valid coordinates → run DBSCAN
    if len(valid_idx) > 0:
        coords_rad = np.radians(coords[valid_mask])

        db = DBSCAN(eps=eps, min_samples=1, metric="haversine")
        loc_labels = db.fit_predict(coords_rad)

        # assign global unique cluster ids
        for lbl in np.unique(loc_labels):
            df.loc[valid_idx[loc_labels == lbl], "city_loc_cluster"] = cluster_counter
            cluster_counter += 1

    # For invalid/missing coords → each row its own cluster (still city-specific)
    for idx in invalid_idx:
        df.loc[idx, "city_loc_cluster"] = cluster_counter
        cluster_counter += 1

# =============================
# 3. FUZZY GROUPING INSIDE EACH CITY+LOCATION CLUSTER
# =============================

df["final_store_group_key"] = None

for cluster_id in df["city_loc_cluster"].unique():
    sub = df[df["city_loc_cluster"] == cluster_id]

    unique_clean_names = sub["clean_name"].dropna().unique()
    group_map = {}

    for name in unique_clean_names:
        if name not in group_map:
            group_map[name] = name  # representative
            for other in unique_clean_names:
                if other not in group_map:
                    score = fuzz.token_sort_ratio(name, other)
                    if score >= 85:  # can tune
                        group_map[other] = name

    mask = df["city_loc_cluster"] == cluster_id
    df.loc[mask, "final_store_group_key"] = df.loc[mask, "clean_name"].map(group_map)

# =============================
# 4. CREATE CANONICAL NAME (STORE + CITY)
# =============================


import numpy as np

# Drop groups where Store Name is completely missing
valid_df = df.dropna(subset=["Store Name"])

def pick_canonical(x):
    # x is a Series of Store Name values
    vc = x.value_counts()
    if len(vc) == 0:
        return np.nan
    return vc.index[0]   # most frequent name

canonical = (
    valid_df
    .groupby(["clean_city", "final_store_group_key"])["Store Name"]
    .agg(pick_canonical)
    .rename("canonical_store_name")
    .reset_index()
)

df = df.merge(canonical, on=["clean_city", "final_store_group_key"], how="left")

# Combined label
df["canonical_store_city"] = df["canonical_store_name"].fillna("").astype(str) + " - " + df["City"].fillna("").astype(str)






#kunal


# =============================
# 5. FINAL MAPPING VIEW
# =============================
# -- 	LIS Store Code

mapping_df = (
    df[[
        "LIS Store Code",
        "Store Name",
        "City",
        "Latitude",
        "Longitude",
        "clean_city",
        "clean_name",
        "city_loc_cluster",
        "final_store_group_key",
        "canonical_store_name",
        "canonical_store_city"
    ]]
    .drop_duplicates()
    .sort_values(["canonical_store_city", "Store Name"])
)

# mapping_df.to_csv("store_group_mapping_with_city.csv", index=False)

print(mapping_df.head(20))


             LIS Store Code                                    Store Name  \
11554         GCPL-SAMT-934                                           NaN   
2754       Parle-Store-1680                             NILGIRIS -OKKIYAM   
15642      Parle-Store-1143                                      THRISSUR   
15673     Parle-StoreS-1040                                      THRISSUR   
11038         HAL-Store-328   1 Daily Needs (Cbd Svs)-D-D-D-D-D-D (62279)   
11051        HAL-Store-3340  1 Daily Needs (Seawood)-D-D-D-D-D-D (367519)   
4727                MCR-791                                      10 Pluss   
3337      1043114010_N00723                                     11 7 MART   
8751       Parle-StoreW-574                                     11/7 MART   
5692   DBRPH-MRG0GO00060812                             1ST CHOICE STORES   
4582               MCR-1657                               1st Beauty Fair   
12511               MCR-114                                       2 Fresh   

In [70]:
df.to_csv('keyword_output_citylatlong.csv')